<a href="https://colab.research.google.com/github/hejnal/kschool-marketing-digital-geo-bqml/blob/main/notebooks/solutions/exercise2_EDA_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA Analysis for Spotify Dataset

## Instructions

Use Jupyter notebook and standard libraries to analyze the data and generate graphs.

To offload the memory consumption, BigQuery DataFrames can be used instead of normal Pandas.

## Install and import Libraries

In [ ]:
!pip install --user --upgrade --quiet bigframes plotly yellowbrick scikit-learn

In [ ]:
import seaborn as sns
from yellowbrick.target import FeatureCorrelation
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

sns.set(rc={'figure.figsize':(11.7,8.27)})

## [Colab Only] Authenticate

In [ ]:
from google.colab import auth
auth.authenticate_user()

## Setup Project and Region and Table Name

In [ ]:
import bigframes.pandas as bpd

PROJECT_ID = "clean-silo-405314"  # @param {type:"string"}
REGION = "US"  # @param {type:"string"}
bpd.close_session()

# Set BigQuery DataFrames options
# Note: The project option is not required in all environments.
# On BigQuery Studio, the project ID is automatically detected.
bpd.options.bigquery.project = PROJECT_ID

# Note: The location option is not required.
# It defaults to the location of the first table or query
# passed to read_gbq(). For APIs where a location can't be
# auto-detected, the location defaults to the "US" location.
bpd.options.bigquery.location = REGION

## Load data directly from BigQuery, using magic bigquery functions or BigQuery DataFrames

### BigFrames option - all aggregations are done in BigQuery

In [ ]:
df = bpd.read_gbq('raw_data.spotify_full_dataset', columns=["artist_name", "track_name", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "key", "liveness", "loudness", "mode", "popularity", "speechiness", "tempo", "valence", "year"], use_cache=False)


### Magic Keyword option - download data to Pandas, process data in the local memory

In [ ]:
# Load BigQuery Magic extension
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery df --project $PROJECT_ID --no_query_cache
SELECT
  artist_name,
  track_name,
  popularity,
  year,
  genre,
  danceability,
  energy,
  key,
  loudness,
  mode,
  speechiness,
  acousticness,
  instrumentalness,
  liveness,
  valence,
  tempo,
  duration_ms,
  time_signature
FROM
  `raw_data.spotify_full_dataset`

## Explore Spotify dataset

### Describe the dataframe

In [ ]:
df.describe()

### Inspect the data

In [ ]:
# show first 5 rows
df.head()

Let's check for the null values

In [ ]:
df.isnull().sum()

Let's see the stats for all the features

In [ ]:
df_stats = df.describe()
df_stats = df_stats.transpose()
df_stats

In [ ]:
df.dtypes

### Histograms

Let's see the popularity charts for artists after 2010.

In [ ]:
df_filtered = df.loc[df['year'] > 2010]

sns.set(rc={'figure.figsize':(14.7,8.27)})
sns.histplot(df_filtered['popularity'], kde=False)

Popularity without outliers.

In [ ]:
from scipy import stats
numeric_features = df.select_dtypes(np.number)
numeric_features_filtered = numeric_features.loc[(numeric_features['year'] >= 2010) & (numeric_features['popularity'] > 0)]
numeric_features_filtered['popularity']
np.abs(stats.zscore(np.array(numeric_features_filtered['popularity'], dtype=np.float64)))

numeric_features_with_no_outliers = numeric_features_filtered[(np.abs(stats.zscore(np.array(numeric_features_filtered['popularity'], dtype=np.float64))) < 3)]

sns.set(rc={'figure.figsize':(14.7,8.27)})
sns.histplot(numeric_features_with_no_outliers['popularity'], kde=False)

Analyse the last 3 years.

In [ ]:
df_filtered = df.loc[(df['year'] >= 2020) & (df['year'] <= 2023)]

Analyse the number of songs per decade.

In [ ]:
def get_decade(year):
    period_start = int(year/10) * 10
    decade = '{}s'.format(period_start)
    return decade

df['decade'] = df['year'].apply(get_decade)

sns.displot(df['decade'])

### Correlation between features

In [ ]:
# Filter to numeric columns
numeric_columns = df.select_dtypes(include=np.number).columns
df_numeric = df[numeric_columns]

sns.set(rc={'figure.figsize':(12.7,8.27)})
# Calculate correlation and plot heatmap
sns.heatmap(df_numeric.corr())

More advanced correlations: energy and popularity, for different modes - major and minor (blue orange) for each year separately.

In [ ]:
sns.set_theme()
sns.set(rc={'figure.figsize':(12.7,8.27)})
sns.relplot(data=df_filtered, x='energy', y='popularity', height=10, aspect=2, hue='mode', col='year', col_wrap=2)

More basic correlation in the bar chart.

In [ ]:
feature_names = ['acousticness', 'danceability', 'energy', 'instrumentalness',
                 'liveness', 'loudness', 'speechiness', 'tempo', 'valence','duration_ms', 'key', 'mode']

X, y = df[feature_names], df['popularity']

# Convert Int64 columns to float64 as a safe option
for col in ['duration_ms', 'key', 'mode']:
    if col in X.columns and X[col].dtype == 'Int64':
        X[col] = X[col].astype(np.float64) # Use float64 to be consistent

# Create a list of the feature names (already done, but keeping for context)
features = np.array(feature_names)

# Instantiate the visualizer
visualizer = FeatureCorrelation(labels=features)

plt.rcParams['figure.figsize']=(15,15)
visualizer.fit(X, y)     # Fit the data to the visualizer
visualizer.show()


### Timeseries

In [ ]:
numeric_features = df.select_dtypes(np.number)

In [ ]:
features_by_year = numeric_features.groupby("year", as_index=False).mean()
sound_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence']

fig = px.line(features_by_year, x='year', y=sound_features, height=1000, width=1800)
fig.show()

## Exercises
For your favourite artist get some interesting stats about their career, how their song evolves, over time and what makes them successful.

Find ansers to the following questions:

* See the feature evolution over time.
* In which years they published their songs (albums) and how many songs were released?
* What is the most popular song by the artist?
* In which year were the songs with the highest average energy levels released?
* What is the name of the most danceable song by your favorite artist (the one in the group)?
* Which feature has the highest correlation with song popularity?


In [ ]:
# @title Helper Function
# Helper function to compare Pandas and SQL results
def compare_results(pandas_df, sql_df, id_column=None):
    """
    Compare basic DataFrames from pandas and SQL with minimal assumptions.

    Args:
        pandas_df: DataFrame from pandas operation
        sql_df: DataFrame from SQL query
        id_column: Column to sort by for comparison (like 'year', 'track_name', etc.)
    """
    # 1. Check if shapes match
    if pandas_df.shape != sql_df.shape:
        print(f"Shape mismatch: Pandas {pandas_df.shape}, SQL {sql_df.shape}")
        return False

    # 2. Ensure columns match (ignoring order)
    pandas_cols = set(pandas_df.columns)
    sql_cols = set(sql_df.columns)
    if pandas_cols != sql_cols:
        print(f"Column mismatch: Only in Pandas {pandas_cols - sql_cols}, Only in SQL {sql_cols - pandas_cols}")
        return False

    # 3. Sort both DataFrames if possible
    if id_column and id_column in pandas_df.columns and id_column in sql_df.columns:
        pandas_df = pandas_df.sort_values(id_column).reset_index(drop=True)
        sql_df = sql_df.sort_values(id_column).reset_index(drop=True)
    else:
        # If no id_column, sort by all columns
        pandas_df = pandas_df.sort_values(list(pandas_df.columns)).reset_index(drop=True)
        sql_df = sql_df.sort_values(list(sql_df.columns)).reset_index(drop=True)

    # 4. Display the first few rows of each for visual inspection
    print("Pandas DataFrame:")
    display(pandas_df.head())
    print("\nSQL DataFrame:")
    display(sql_df.head())

    # 5. Check if values are approximately equal (handle floating point)
    for col in pandas_df.columns:
        pandas_col = pandas_df[col]
        sql_col = sql_df[col]

        # Convert to same type for comparison
        if pandas_col.dtype != sql_col.dtype:
            # Try to convert both to string for comparison
            pandas_str = pandas_col.astype(str)
            sql_str = sql_col.astype(str)

            if not (pandas_str == sql_str).all():
                print(f"Values differ in column: {col}")
                return False

    print("✅ Dataframe comparison passed!")
    return True

### Exercise 1: Filter Data by Artist

In [ ]:
df.loc[df["artist_name"] == "Bon Iver"]

In [ ]:
# @title Pandas Solution
my_artist_df_pandas = df.loc[df["artist_name"] == "Bon Iver"]

In [ ]:
my_artist_df_pandas

In [ ]:
# @title SQL Exercise
# TODO: Filter the dataset by artist using SQL (BigQuery)
# Use the %%bigquery magic command to query the raw_data.spotify_full_dataset table

%%bigquery my_artist_df --project $PROJECT_ID


In [ ]:
# @title SQL Solution
%%bigquery my_artist_df_sql --project $PROJECT_ID
SELECT *
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'

In [ ]:
# @title Compare Pandas vs SQL
# Compare the results of Pandas and SQL filtering
common_columns = list(set(my_artist_df_pandas.columns) & set(my_artist_df_sql.columns))
compare_results(my_artist_df_pandas[common_columns], my_artist_df_sql[common_columns])

### Exercise 2: How song features have evolved throughout the years?

In [ ]:
# @title Pandas Exercise
# TODO: Calculate the feature evolution over time using Pandas
# Group by year and calculate the mean of numeric features
features_by_year_pandas =

In [ ]:
# @title Pandas Solution
# Extract numeric features from the artist dataframe
numeric_features = my_artist_df_pandas.select_dtypes(np.number)
# Group by year and calculate the mean of each feature
features_by_year_pandas = numeric_features.groupby("year", as_index=False).mean()

In [ ]:
# @title SQL Exercise
# TODO: Calculate the feature evolution over time using SQL (BigQuery)
# Write a query that groups by year and calculates the average of each sound feature

%%bigquery features_by_year_sql --project $PROJECT_ID


In [ ]:
# @title SQL Solution
%%bigquery features_by_year_sql --project $PROJECT_ID
SELECT
  year,
  AVG(acousticness) AS acousticness,
  AVG(danceability) AS danceability,
  AVG(energy) AS energy,
  AVG(instrumentalness) AS instrumentalness,
  AVG(liveness) AS liveness,
  AVG(valence) AS valence,
  AVG(loudness) AS loudness,
  AVG(speechiness) AS speechiness,
  AVG(tempo) AS tempo,
  AVG(duration_ms) AS duration_ms,
  AVG(key) AS key,
  AVG(mode) AS mode,
  AVG(popularity) AS popularity
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
GROUP BY year
ORDER BY year

In [ ]:
# @title Compare Pandas vs SQL
# Compare Pandas and SQL results for feature evolution
# Focus on key sound features for comparison
sound_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence']
# Add 'year' to comparison columns since it's our grouping key
compare_columns = ['year'] + sound_features
compare_results(features_by_year_pandas[compare_columns], features_by_year_sql[compare_columns])

In [ ]:
# Visualize the feature evolution over time
sound_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence']
fig = px.line(features_by_year_sql, x='year', y=sound_features, height=1000, width=1800,
              title="Evolution of Sound Features Over Time for Bon Iver")
fig.show()

### Exercise 3: In which years they published their songs (albums) and how many songs were released?

In [ ]:
# @title Pandas Exercise
# TODO: In which years they published their songs (albums) and how many songs were released? (use num_songs as the num_songs count)

songs_by_year_df_pandas =

In [ ]:
# @title Pandas Solution
# TODO: In which years they published their songs (albums) and how many songs were released? (use num_songs as the num_songs count)

songs_by_year_df_pandas = my_artist_df_pandas.groupby("year").size().reset_index(name='num_songs').sort_index(ascending=True)

In [ ]:
# @title SQL Exercise
# Use the %%bigquery magic command to query the raw_data.spotify_full_dataset table
# (use num_songs as the num_songs count)

%%bigquery songs_by_year_df_sql --project $PROJECT_ID


In [ ]:
# @title SQL Solution
%%bigquery songs_by_year_df_sql --project $PROJECT_ID
SELECT
  year,
  COUNT(*) AS num_songs
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
GROUP BY year
ORDER BY year

In [ ]:
# @title Compare Pandas vs SQL
# Compare the results of Pandas and SQL filtering
common_columns = list(set(songs_by_year_df_pandas.columns) & set(songs_by_year_df_sql.columns))
compare_results(songs_by_year_df_pandas[common_columns], songs_by_year_df_sql[common_columns])

### Exercise 4: What is the most popular song of your artist?

In [ ]:
# @title Pandas Exercise
# TODO: What is the most popular song by the artist?
most_popular_song_df_pandas = # TODO: use nlargest() function


In [ ]:
# @title Pandas Solution
most_popular_song_df_pandas = my_artist_df_pandas.nlargest(1, 'popularity')

In [ ]:
# @title SQL Exercise
# TODO: Filter the dataset by artist using SQL (BigQuery)
# Use the %%bigquery magic command to query the raw_data.spotify_full_dataset table

%%bigquery most_popular_song_df_sql --project $PROJECT_ID

In [ ]:
# @title SQL Solution
%%bigquery most_popular_song_df_sql --project $PROJECT_ID
SELECT
  *
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
ORDER BY popularity DESC
LIMIT 1

In [ ]:
most_popular_song_df_sql

In [ ]:
# @title Compare Pandas vs SQL
# Compare the results of Pandas and SQL filtering
common_columns = list(set(most_popular_song_df_pandas.columns) & set(most_popular_song_df_sql.columns))
compare_results(most_popular_song_df_pandas[common_columns], most_popular_song_df_sql[common_columns])

### Exercise 5: What has been the year with the highest energy?

In [ ]:
# @title Pandas Exercise
# TODO: In which year were the songs with the highest average energy levels released?

# use groupby and agg() function.
highest_energy_df_pandas =

In [ ]:
# @title Pandas Solution
# TODO: In which year were the songs with the highest average energy levels released?
highest_energy_df_pandas = my_artist_df_pandas.groupby("year").agg({"energy": "mean"}).sort_values(by="energy", ascending=False).head(1).rename(columns={"energy": "avg_energy"})

In [ ]:
# @title SQL Exercise
# TODO: Filter the dataset by artist using SQL (BigQuery)
# Use the %%bigquery magic command to query the raw_data.spotify_full_dataset table

%%bigquery highest_energy_df_sql --project $PROJECT_ID


In [ ]:
# @title SQL Solution
%%bigquery highest_energy_df_sql --project $PROJECT_ID

SELECT
  year,
  AVG(energy) AS avg_energy
FROM `raw_data.spotify_full_dataset`
WHERE artist_name = 'Bon Iver'
GROUP BY year
ORDER BY avg_energy DESC
LIMIT 1

In [ ]:
# @title Compare Pandas vs SQL
# Compare the results of Pandas and SQL filtering
common_columns = list(set(highest_energy_df_pandas.columns) & set(highest_energy_df_sql.columns))
compare_results(highest_energy_df_pandas[common_columns], highest_energy_df_sql[common_columns])

### Exercise 6: Feature Correlation with Popularity

In [ ]:
# @title Pandas Exercise
# TODO: Which feature has the highest correlation with song popularity?

# Filter to numeric columns
df_bon_iver_numeric_columns = my_artist_df_pandas.select_dtypes(include=np.number).columns
df_bon_iver_numeric = my_artist_df_pandas[df_bon_iver_numeric_columns]

# use corr() function. ignore popularity and year indexes, sort and limit 1
feature_corr_with_target_df_pandas =

In [ ]:
# @title Pandas Solution
# TODO: Which feature has the highest correlation with song popularity?

# Filter to numeric columns
df_bon_iver_numeric_columns = my_artist_df_pandas.select_dtypes(include=np.number).columns
df_bon_iver_numeric = my_artist_df_pandas[df_bon_iver_numeric_columns]

# First calculate the correlation matrix
correlation_matrix = df_bon_iver_numeric.corr()

# Extract just the popularity column and drop popularity itself and any other columns you want to exclude
feature_correlations = correlation_matrix.drop(['popularity', 'year'], axis=0)[['popularity']]

# Add an absolute correlation column for sorting
feature_correlations['abs_correlation'] = feature_correlations['popularity'].abs()

# Sort by absolute correlation (descending) and get top feature
top_feature = feature_correlations.sort_values(by='abs_correlation', ascending=False).head(1)

# Reset index to make the feature name a column
feature_corr_with_target_df_pandas = top_feature.reset_index()

# Rename the index column to 'feature'
feature_corr_with_target_df_pandas.rename(columns={'index': 'feature'}, inplace=True)

# If needed, drop the abs_correlation column to match SQL output
feature_corr_with_target_df_pandas = feature_corr_with_target_df_pandas[['feature', 'popularity']]

# Rename popularity to match SQL output
feature_corr_with_target_df_pandas.rename(columns={'popularity': 'correlation_with_popularity'}, inplace=True)


In [ ]:
feature_corr_with_target_df_pandas

In [ ]:
# @title SQL Exercise
# TODO: Filter the dataset by artist using SQL (BigQuery)
# Use the %%bigquery magic command to query the raw_data.spotify_full_dataset table

%%bigquery my_artist_df --project $PROJECT_ID


In [ ]:
# @title SQL Solution
%%bigquery feature_corr_with_target_df_sql --project $PROJECT_ID

WITH all_correlations AS (
  SELECT
    'danceability' AS feature,
    CORR(danceability, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'energy' AS feature,
    CORR(energy, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'acousticness' AS feature,
    CORR(acousticness, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'instrumentalness' AS feature,
    CORR(instrumentalness, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'liveness' AS feature,
    CORR(liveness, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'valence' AS feature,
    CORR(valence, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'tempo' AS feature,
    CORR(tempo, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'loudness' AS feature,
    CORR(loudness, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'speechiness' AS feature,
    CORR(speechiness, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'duration_ms' AS feature,
    CORR(duration_ms, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'key' AS feature,
    CORR(key, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'

  UNION ALL

  SELECT
    'mode' AS feature,
    CORR(mode, popularity) AS correlation_with_popularity
  FROM `raw_data.spotify_full_dataset`
  WHERE artist_name = 'Bon Iver'
)

SELECT
  feature,
  correlation_with_popularity
FROM all_correlations
ORDER BY ABS(correlation_with_popularity) DESC
LIMIT 1

In [ ]:
# @title Compare Pandas vs SQL
# Compare the results of Pandas and SQL filtering
common_columns = list(set(feature_corr_with_target_df_pandas.columns) & set(feature_corr_with_target_df_sql.columns))
compare_results(feature_corr_with_target_df_pandas[common_columns], feature_corr_with_target_df_sql[common_columns])